In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from sklearn.metrics import pairwise_distances

In [2]:
def cluster_data(data,n_clusters=128):
    kmeans=KMeans(n_clusters=128,random_state=0,verbose=1).fit(data)
    cluster_centers=kmeans.cluster_centers_
    np.save('./saved_clusters/cluster_centers.npy',cluster_centers)
    cluster_labels=kmeans.labels_
    np.save('./saved_clusters/cluster_labels.npy',cluster_labels)  


def select_clusters(cluster_means,covid_data,per=0.1):
    
    Map=pairwise_distances(cluster_means,covid_data)
    A=np.argsort(Map,axis=0)
    A=A[0,:]

    (unique, counts) = np.unique(A, return_counts=True)
    ind=np.argsort(counts)
    selected_clusters=unique[ind]
    
    thresh=int(per*cluster_means.shape[0])

    if thresh>len(selected_clusters):
       final_selection=selected_clusters
    else:
       final_selection=selected_clusters[0:thresh]   
    
    return final_selection # index of clusters selected 



def select_training_data_from_clusters(selected_cluster_index,clustering_labels,train):    
    train['clustering_labels']=clustering_labels
    B=train['clustering_labels']
    C=B[B.isin(selected_cluster_index)] 
    C=C.index.to_numpy()

    data=train.iloc[C,:]
    data=data.drop(columns=['clustering_labels'])
    
    return data # subset of training data 



def ICD_codes_selection_five_conditions(path_icd_files,selcted_data):
   
    B=selcted_data['hadm_id']
    codes=listdir(path_icd_files)

    ICD=[]

    for c in codes:
        print(c)
        df=pd.read_csv(path_icd_files+c)
        A=df['ID']
        C=A[A.isin(B)]
        C=C.index.to_numpy()
        ICD.append(df.iloc[C,1].to_numpy())
   
    ICD=np.hstack(ICD)
    
    return ICD


def ICD_codes_selection(icd_file,selcted_data):
   
    B=selcted_data['hadm_id']
    A=icd_file['ID']
    C=A[A.isin(B)]
    C=C.index.to_numpy()
    ICD=icd_file.iloc[C,1].to_numpy()
 
    return ICD